### TensorFlow Addons Networks : Sequence-to-Sequence NMT with Attention Mechanism

In [1]:
!wget --quiet http://www.manythings.org/anki/deu-eng.zip
!unzip deu-eng.zip

Archive:  deu-eng.zip
  inflating: deu.txt                 
  inflating: _about.txt              


In [2]:
!pip install tensorflow-addons

In [3]:
import re
import csv
import string
import random
import itertools

import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa

from pickle import load, dump
from typing import List, Tuple
from unicodedata import normalize
from keras.models import load_model
from keras.utils.vis_utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from nltk.translate.bleu_score import corpus_bleu
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, LSTM, Embedding

### Data Preprocessing

In [4]:
# Start of sentence
SOS = "<start>"
# End of sentence
EOS = "<end>"
# Relevant punctuation
PUNCTUATION = set("?,!.")


def load_dataset(filename: str) -> str:
    """
    load dataset into memory
    """
    with open(filename, mode = "rt", encoding = "utf-8") as fp:
        return fp.read()


def to_pairs(dataset: str, limit: int = None, shuffle = False) -> List[Tuple[str, str]]:
    """
    Split dataset into pairs of sentences, discards dataset line info.

    e.g.
    input -> 'Go.\tGeh.\tCC-BY 2.0 (France) Attribution: tatoeba.org
    #2877272 (CM) & #8597805 (Roujin)'
    output -> [('Go.', 'Geh.')]

    :param dataset: dataset containing examples of translations between
    two languages
    the examples are delimited by `\n` and the contents of the lines are
    delimited by `\t`
    :param limit: number that limit dataset size (optional)
    :param shuffle: default is True
    :return: list of pairs
    """
    assert isinstance(limit, (int, type(None))), TypeError(
        "the limit value must be an integer"
    )
    lines = dataset.strip().split("\n")
    # Radom dataset
    if shuffle is True:
        random.shuffle(lines)
    number_examples = limit or len(lines)  # if None get all
    pairs = []
    for line in lines[: abs(number_examples)]:
        # take only source and target
        src, trg, _ = line.split("\t")
        pairs.append((src, trg))

    # dataset size check
    assert len(pairs) == number_examples
    return pairs


def separe_punctuation(token: str) -> str:
    """
    Separe punctuation if exists
    """

    if not set(token).intersection(PUNCTUATION):
        return token
    for p in PUNCTUATION:
        token = f" {p} ".join(token.split(p))
    return " ".join(token.split())


def preprocess(sentence: str, add_start_end: bool=True) -> str:
    """

    - convert lowercase
    - remove numbers
    - remove special characters
    - separe punctuation
    - add start-of-sentence <start> and end-of-sentence <end>

    :param add_start_end: add SOS (start-of-sentence) and EOS (end-of-sentence)
    """
    re_print = re.compile(f"[^{re.escape(string.printable)}]")
    # convert lowercase and normalizing unicode characters
    sentence = (
        normalize("NFD", sentence.lower()).encode("ascii", "ignore").decode("UTF-8")
    )
    cleaned_tokens = []
    # tokenize sentence on white space
    for token in sentence.split():
        # removing non-printable chars form each token
        token = re_print.sub("", token).strip()
        # ignore tokens with numbers
        if re.findall("[0-9]", token):
            continue
        # add space between words and punctuation eg: "ok?go!" => "ok ? go !"
        token = separe_punctuation(token)
        cleaned_tokens.append(token)

    # rebuild sentence with space between tokens
    sentence = " ".join(cleaned_tokens)

    # adding a start and an end token to the sentence
    if add_start_end is True:
        sentence = f"{SOS} {sentence} {EOS}"
    return sentence


def dataset_preprocess(dataset: List[Tuple[str, str]]) -> Tuple[List[str], List[str]]:
    """
    Returns processed database

    :param dataset: list of sentence pairs
    :return: list of paralel data e.g. 
    (['first source sentence', 'second', ...], ['first target sentence', 'second', ...])
    """
    source_cleaned = []
    target_cleaned = []
    for source, target in dataset:
        source_cleaned.append(preprocess(source))
        target_cleaned.append(preprocess(target))
    return source_cleaned, target_cleaned

### Create Dataset
- limit number of examples
- load dataset into pairs ```[('Be nice.', 'Seien Sie nett!'), ('Beat it.', 'Geh weg!'), ...]```
- preprocessing dataset

In [5]:
NUM_EXAMPLES = 10000 

filename = 'deu.txt'
dataset = load_dataset(filename)

pairs = to_pairs(dataset, limit = NUM_EXAMPLES)
print(f"Dataset size: {len(pairs)}")
raw_data_en, raw_data_ge = dataset_preprocess(pairs)

# show last 5 pairs
for pair in zip(raw_data_en[-5:],raw_data_ge[-5:]):
    print(pair)

Dataset size: 10000
('<start> tom was crying . <end>', '<start> tom flennte . <end>')
('<start> tom was eating . <end>', '<start> tom hat gegessen . <end>')
('<start> tom was famous . <end>', '<start> tom war beruhmt . <end>')
('<start> tom was framed . <end>', '<start> tom wurde reingelegt . <end>')
('<start> tom was fuming . <end>', '<start> tom war wutend . <end>')


### Tokenization

In [6]:
en_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters = '')
en_tokenizer.fit_on_texts(raw_data_en)

data_en = en_tokenizer.texts_to_sequences(raw_data_en)
data_en = tf.keras.preprocessing.sequence.pad_sequences(data_en,padding = 'post')

ge_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters = '')
ge_tokenizer.fit_on_texts(raw_data_ge)

data_ge = ge_tokenizer.texts_to_sequences(raw_data_ge)
data_ge = tf.keras.preprocessing.sequence.pad_sequences(data_ge,padding = 'post')

In [7]:
def max_len(tensor):
    return max( len(t) for t in tensor)

### Training Prep

In [8]:
X_train,  X_test, Y_train, Y_test = train_test_split(data_en,data_ge,test_size = 0.2)
BATCH_SIZE = 64
BUFFER_SIZE = len(X_train)
steps_per_epoch = BUFFER_SIZE//BATCH_SIZE
embedding_dims = 256
rnn_units = 1024
dense_units = 1024
Dtype = tf.float32   #used to initialize DecoderCell Zero state

Tx = max_len(data_en)
Ty = max_len(data_ge)  

input_vocab_size = len(en_tokenizer.word_index)+1  
output_vocab_size = len(ge_tokenizer.word_index)+ 1
dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True)
example_X, example_Y = next(iter(dataset))
print(example_X.shape) 
print(example_Y.shape) 

(64, 9)
(64, 13)


### Build Model

In [9]:
# ENCODER
class EncoderNetwork(tf.keras.Model):
    def __init__(self,input_vocab_size,embedding_dims, rnn_units ):
        super().__init__()
        self.encoder_embedding = tf.keras.layers.Embedding(input_dim = input_vocab_size,
                                                           output_dim = embedding_dims)
        self.encoder_rnnlayer = tf.keras.layers.LSTM(rnn_units,return_sequences = True, 
                                                     return_state = True )
    
# DECODER
class DecoderNetwork(tf.keras.Model):
    def __init__(self,output_vocab_size, embedding_dims, rnn_units):
        super().__init__()
        self.decoder_embedding = tf.keras.layers.Embedding(input_dim = output_vocab_size,
                                                           output_dim = embedding_dims) 
        self.dense_layer = tf.keras.layers.Dense(output_vocab_size)
        self.decoder_rnncell = tf.keras.layers.LSTMCell(rnn_units)
        # Sampler
        self.sampler = tfa.seq2seq.sampler.TrainingSampler()
        # Create attention mechanism with memory = None
        self.attention_mechanism = self.build_attention_mechanism(dense_units,None,BATCH_SIZE*[Tx])
        self.rnn_cell =  self.build_rnn_cell(BATCH_SIZE)
        self.decoder = tfa.seq2seq.BasicDecoder(self.rnn_cell, sampler = self.sampler,
                                                output_layer = self.dense_layer)

    def build_attention_mechanism(self, units,memory, memory_sequence_length):
        return tfa.seq2seq.LuongAttention(units, memory = memory, 
                                          memory_sequence_length = memory_sequence_length)

    # wrap decodernn cell  
    def build_rnn_cell(self, batch_size ):
        rnn_cell = tfa.seq2seq.AttentionWrapper(self.decoder_rnncell, self.attention_mechanism,
                                                attention_layer_size=dense_units)
        return rnn_cell
    
    def build_decoder_initial_state(self, batch_size, encoder_state, Dtype):
        decoder_initial_state = self.rnn_cell.get_initial_state(batch_size = batch_size, 
                                                                dtype = Dtype)
        decoder_initial_state = decoder_initial_state.clone(cell_state = encoder_state) 
        return decoder_initial_state

encoderNetwork = EncoderNetwork(input_vocab_size,embedding_dims, rnn_units)
decoderNetwork = DecoderNetwork(output_vocab_size,embedding_dims, rnn_units)
optimizer = tf.keras.optimizers.Adam()